# <font color="green"> BEMM459 - Week 7 - ORM with SQLAlchemy </font>
SQLAlchemy is the Python SQL toolkit and <b>Object Relational Mapper (ORM)</b> that gives application developers the full power and flexibility of SQL. 
Ack of resources used for the tutorial: 
www.sqlalchemy.org/; www.tutorialspoint.com

In [ ]:
# Import sqlalchemy
import sqlalchemy

# Check version
sqlalchemy.__version__ 

# 1. Declare Mapping
### The main objective of the Object Relational Mapper API of SQLAlchemy is to facilitate associating user-defined Python classes with database tables, and objects of those classes with rows in their corresponding tables. 
### Changes in states of objects and rows are synchronously matched with each other. SQLAlchemy enables expressing database queries in terms of user defined classes and their defined relationships.

In [ ]:
# Imports the declarative_base object, which connects the database engine to the SQLAlchemy functionality of the models.
# The declarative base class stores a catlog of classes and mapped tables in the Declarative system.
from sqlalchemy.ext.declarative import declarative_base

# Imports the Column, Integer, String classes from SQLAlchemy, which are used to help define the model attributes.
from sqlalchemy import Column, Integer, String

# Imports create_engine
from sqlalchemy import create_engine

# create_engine() function is called to set up an engine object which is subsequently used to perform SQL operations. 
# Create an engine that stores data in the local directory. "///" is the relative path (The SQLite database is created in the same folder as the Jupyter script)
# The first parameter is the name of the database.
# The second parameter (echo) is set to true; this will generate the activity log
engine = create_engine('sqlite:///Week7_NoSQL.db', echo = True)

# Creates the Base class, which is what all models inherit from and how they get SQLAlchemy ORM functionality.
# Once base class is declared, any number of mapped classes can be defined in terms of it (a reference is passed to the class).
Base=declarative_base()

# Defines the Student class model to the customers database table.
# In ORM, the configurational process starts by describing the database tables and then by defining classes which will be mapped to those tables. 
# In SQLAlchemy, these two tasks are performed together. 
# A class in Declarative must have a __tablename__ attribute, and at least one Column which is part of a primary key. 
class Student(Base):
    __tablename__ = 'students'
    stud_id = Column(Integer, primary_key=True)
    stud_name = Column(String)
    stud_address = Column(String)
    stud_email = Column(String)

# Each Table object is a member of larger collection known as MetaData and this object is available using the .metadata attribute of declarative base class. 
# The MetaData.create_all() method is, passing in our Engine as a source of database connectivity. For all tables that haven’t been created yet, it issues CREATE TABLE statements to the database.
# Create all tables in the engine. This is equivalent to "Create Table"
Base.metadata.create_all(engine)

# 2. Creating Session and adding records to the SQLite database

In [ ]:
# The Session object enables us to interest with the database (it is a handle to the database)
from sqlalchemy.orm import sessionmaker

# Session class is defined using sessionmaker(), which is bound to the engine object created earlier
Session = sessionmaker(bind = engine)

# session object (objSession) is set-up using the default constructor
objSession = Session()

# We have declared a Student class and which has been mapped to the 'students' table
# We create an object of the Student class and then add it to the table 'students' by using the add() method of the session object
student1 = Student(stud_name = 'Hello World', stud_address = 'Flat 1, East Lane, Exeter EX3 5TY', stud_email = 'hello.world@exeter.ac.uk')

# use add() method of the Session class used to add one record
objSession.add(student1)

# Flushes all items and any transaction in progress
# The student object is added to the table 'students'
objSession.commit()

In [ ]:
# use add_all() method of the Session class used to add multiple record
# open database in SQLIte and check if records are added (use ".mode column" and ".header on" options for formatting)
objSession.add_all([
   Student(stud_name = 'Hello SQLite', stud_address = 'Flat 12, East Park, Exeter EX3 6YU', stud_email = 'Hello.SQLite@exeter.ac.uk'), 
   Student(stud_name = 'Hello Redis', stud_address = '56 Pennsylvania Road, Exeter EX4 7YU', stud_email = 'Hello.Redis@exeter.ac.uk'), 
   Student(stud_name = 'Hello Mongo', stud_address = '111, St. Thomas Park, Exeter EX2 9HT', stud_email = 'Hello.Mongo@exeter.ac.uk')]
)
objSession.commit()

# 3. Query records using SQLAlchemy ORM object

In [ ]:
# SELECT statements generated by SQLAlchemy ORM are constructed by Query object
# Query objects are initially generated using the query() method of the Session. e.g., q = session.query(mapped class) or q = Query(mappedClass, session)

# query object has all() method which returns a resultset in the form of list of objects
resultSet = objSession.query(Student).all()

# Displaying all records
for record in resultSet:
    print("ID:",record.stud_id, "Name:",record.stud_name, "Address:",record.stud_address, "Email:",record.stud_email)

# 4. Update records using SQLAlchemy ORM object
### example of committ and rollback

In [ ]:
# To update a record in the database, we first assign new value to the object attribute and then commit the changes

# The get(x) method of the Query object returns an object which holds the record associated with the primary key (x)
record = objSession.query(Student).get(4)
# Displaying type of object returned
print(type(record))

# Display content of the object
print ("Before update...")
print ("ID:", record.stud_id, "Name:", record.stud_name, "Address:", record.stud_address, "Email:", record.stud_email)

# Updating name field by assigning a new value to the name attribute of the object
record.stud_name = 'Hello Neo4J'
objSession.commit()

record = objSession.query(Student).get(4)
# Display content of the object
print ("After update...")
print ("ID:", record.stud_id, "Name:", record.stud_name, "Address:", record.stud_address, "Email:", record.stud_email)

In [ ]:
record = objSession.query(Student).get(1)
record.stud_name = 'RELATIONAL DATABASE'
# Display content of the object
print ("After update but before rollback...")
print ("ID:", record.stud_id, "Name:", record.stud_name, "Address:", record.stud_address, "Email:", record.stud_email)

# Note that we are not using committ but rollback

objSession.rollback()
print ("After rollback...")
print ("ID:", record.stud_id, "Name:", record.stud_name, "Address:", record.stud_address, "Email:", record.stud_email)

In [ ]:
# using update() method of the Query object to make changes to several records (all records in our case)
# synchronize_session attribute mentions the strategy to update attributes in the session
objSession.query(Student).update({Student.stud_name:"Mr."+Student.stud_name}, synchronize_session = False)
objSession.commit()

# 5. Applying Filters (similar to the WHERE clause in SQL)

In [ ]:
# Resultset represented by Query object can be subjected to certain criteria by using filter() method. 
# The general usage of filter method is as follows − session.query(class).filter(criteria)

#applying filters - greater than, equal to, not equal to, IN, Like with wildcard character, 
resultSet = objSession.query(Student).filter(Student.stud_id>2)

# resultSet = objSession.query(Student).filter(Student.stud_id==2)

# resultSet = objSession.query(Student).filter(Student.stud_id!=2)

# resultSet = objSession.query(Student).filter(Student.stud_id.in_([1,3]))

# resultSet = objSession.query(Student).filter(Student.stud_address.like('Flat%'))

# Displaying all records
for record in resultSet:
    print("ID:",record.stud_id, "Name:",record.stud_name, "Address:",record.stud_address, "Email:",record.stud_email)

In [ ]:
#applying filter - AND
# Method 1 - putting multiple commas separated criteria in the filter
resultSet = objSession.query(Student).filter(Student.stud_id<3, Student.stud_address.like('Flat%'))

for record in resultSet:
    print("ID:",record.stud_id, "Name:",record.stud_name, "Address:",record.stud_address, "Email:",record.stud_email)

In [ ]:
#applying filter - AND
# Method 2 - using the _() method (import and_)

from sqlalchemy import and_
resultSet = objSession.query(Student).filter(and_(Student.stud_id<3, Student.stud_address.like('Flat%')))

for record in resultSet:
    print("ID:",record.stud_id, "Name:",record.stud_name, "Address:",record.stud_address, "Email:",record.stud_email)

In [ ]:
#applying filter - OR

from sqlalchemy import or_
resultSet = objSession.query(Student).filter(or_(Student.stud_id<3, Student.stud_address.like('%Thomas%')))

for record in resultSet:
    print("ID:",record.stud_id, "Name:",record.stud_name, "Address:",record.stud_address, "Email:",record.stud_email)

In [ ]:
# all() - It returns a list. 
resultSet = objSession.query(Student).all()

for record in resultSet:
    print("ID:",record.stud_id, "Name:",record.stud_name, "Address:",record.stud_address, "Email:",record.stud_email)

In [ ]:
# first() - It applies a limit of one and returns the first result 
result=objSession.query(Student).first()

print("ID:",record.stud_id, "Name:",record.stud_name, "Address:",record.stud_address, "Email:",record.stud_email)

In [ ]:
# Textual SQL
# Literal strings can be used flexibly with Query object by specifying their use with the text()

from sqlalchemy import text

for student in objSession.query(Student).filter(text("stud_id<=2")):
   print(student.stud_name)

In [ ]:
# This code block shows the use of one() method, error handling, requesting input from user and binding parameters with string-based SQL

# Required for errorhanding
from sqlalchemy.orm.exc import NoResultFound, MultipleResultsFound

# Requesting user input
studentNum=input("Enter student number: ")
#studentAdd=input("Enter part of address: ")

try:
    # The one() method of the Query object fetches all rows, and if there is not exactly one it raises an error MultipleResultsFound or NoResultFound
    # To specify bind parameters with string-based SQL, we use a colon; to specify the values, we use the params() method
    student = objSession.query(Student).filter(text("stud_id = :value")).params(value = studentNum).one()
    
    #student = objSession.query(Student).filter(text("stud_address LIKE :value")).params(value = studentAdd).one()

    print(student.stud_name)
    
except NoResultFound:
    print("Results not found")   
    
except MultipleResultsFound:
    print("Multiple results found")   


In [ ]:
#To use an string-based statement for SQL, a text() construct representing a complete statement can be passed to from_statement().

resultSet = objSession.query(Student).from_statement(text("SELECT * FROM students")).all()

#resultSet = objSession.query(Student).from_statement(text("SELECT * FROM students where stud_address LIKE '%Flat%'")).all()

for record in resultSet:
    print("ID:",record.stud_id, "Name:",record.stud_name, "Address:",record.stud_address, "Email:",record.stud_email)

# 6. Building Relationships (one-to-many)

In [ ]:
# Imports the declarative_base object, which connects the database engine to the SQLAlchemy functionality of the models.
from sqlalchemy.ext.declarative import declarative_base

# Imports the Column, Integer, String, ForeignKey, and Table classes from SQLAlchemy, which are used to help define the model attributes.
from sqlalchemy import Column, Integer, String, ForeignKey, Table

# Imports the relationship() object, which are used to create the relationships between objects.
from sqlalchemy.orm import relationship

# Imports create_engine
from sqlalchemy import create_engine

# Creates the Base class, which is what all models inherit from and how they get SQLAlchemy ORM functionality.
Base1=declarative_base()

# One-to-many relationship between customers and invoices table (and by extension, Customer and Invoice class)

# Defines the Customer class -> mapped to the customers database table.
class Customer(Base1):
    __tablename__ = 'customers'
    id = Column(Integer, primary_key = True)
    name = Column(String)
    address = Column(String)
    email = Column(String)

# Defines the Invoice class -> mapped to the invoices database table.
# Invoices class applies ForeignKey construct on custid attribute. 
# relationship(), tells the ORM that the Invoice class should be linked to the Customer class using the attribute Invoice.customer.
# The relationship() uses the foreign key relationships between the two tables to determine the nature of this linkage
class Invoice(Base1):
    __tablename__ = 'invoices'
    id = Column(Integer, primary_key = True)
    custid = Column(Integer, ForeignKey('customers.id'))
    invno = Column(Integer)
    amount = Column(Integer)
    customerR = relationship("Customer", back_populates = "invoicesR")

#An additional relationship() directive is placed on the Customer mapped class under the attribute Customer.invoices. 
# The parameter relationship.back_populates is assigned to refer to the complementary attribute names, ..
# so that each relationship() can make intelligent decision about the same relationship as expressed in reverse. 
# On one side, Invoices.customerR refers to Invoices instance, and on the other side, Customer.invoicesR refers to a list of Customers instances.
Customer.invoicesR = relationship("Invoice", order_by = Invoice.id, back_populates = "customerR")

engine1 = create_engine('sqlite:///Week7_NoSQL.db', echo=True)

Base1.metadata.create_all(engine1)

# 7. Adding data into two related tables with ORM objects

In [ ]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind = engine1)
sessionObj = Session()

# Createing an object of Customer class and assigning values
c1 = Customer(name = "Hello World", address = "South Bank, London NE1", email = "hello.world@exeter.com")

# Check which class the object belongs to (useful command)
print(type(c1))

# Assigning values to c1.invoicesR 
c1.invoicesR = [Invoice(invno = 10, amount = 15000), Invoice(invno = 14, amount = 3850)]

sessionObj.add(c1)
sessionObj.commit()

In [ ]:
# Customer object initialised by providing mapped attribute of invoices in the constructor itself
c2 = Customer(
      name = "Hello Neo4J", 
      address = "Lopez Halls, Exeter University",
      email = "Neo4J@gmail.com",
      invoicesR = [Invoice(invno = 3, amount = 10000), 
      Invoice(invno = 4, amount = 5000)
    ])

sessionObj.add(c2)
sessionObj.commit()

In [ ]:
# Adding a list of objects using add_all() function of session object

rows = [
   Customer(
      name = "Hello Redis", 
      address = "Plymouth Quaeters, Plymouth", 
      email = "Redis@gmail.com", 
      invoicesR = [Invoice(invno = 7, amount = 12000), Invoice(invno = 8, amount = 18500)]),

   Customer(
      name = "Hello Cassandra", 
      address = "L.12, Bristol University Computer Lab, Bristol BS1 6YU", 
      email = "Cassandra@gmail.com",
      invoicesR = [Invoice(invno = 9, amount = 15000), 
      Invoice(invno = 11, amount = 6000)
   ])
]

sessionObj.add_all(rows)
sessionObj.commit()

# 8. Working with Joins

In [ ]:
# To construct a simple implicit join between Customer and Invoice, we can use Query.filter() to equate their related columns together

print("\nMethod 1..\n")
result = sessionObj.query(Customer).join(Invoice).filter(Customer.id == Invoice.custid)
for row in result:
    for inv in row.invoicesR:
        print (row.id, row.name, inv.invno, inv.amount)
        

print("\nMethod 2..\n")
for c, i in sessionObj.query(Customer, Invoice).filter(Customer.id == Invoice.custid).all():
    print ("ID: {} Name: {} Invoice No: {} Amount: {}".format(c.id, c.name, i.invno, i.amount))

# 9. Deleting Objects

In [ ]:
# To delete record from a single table, you have to delete an object of the mapped class from a session and commit the action.
recordSet = sessionObj.query(Customer).get(2)
print (recordSet.id, recordSet.name)

# Deleteing one record using .delete() method (pass object with primary key value = 2)
sessionObj.delete(recordSet)

# Confirm to see if deleted (count will be zero)
sessionObj.query(Customer).filter_by(id = 2).count()

# However, related records present in Invoices
sessionObj.query(Invoice).filter(Invoice.invno.in_([10,14])).count()

In [ ]:
# DELETE RELATED RECORD FROM CHILD TABLE (CASCADE DELETE IN SQLite)

# The cascade attribute in relationship function is a comma-separated list of cascade rules which determines how Session operations should be “cascaded” from parent to child. 
# By default: it is False, which means that it is "save-update, merge".
# Change to: "all, delete-orphan" to indicate that related objects should follow along with the parent object in all cases, and be deleted when de-associated.

from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, ForeignKey, Table
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine

# Creates the Base class, which is what all models inherit from and how they get SQLAlchemy ORM functionality.
Base2=declarative_base()

# Creates the Base class, which is what all models inherit from and how they get SQLAlchemy ORM functionality.

class CustomerX(Base2):
    __tablename__ = 'customersX'
    id = Column(Integer, primary_key = True)
    name = Column(String)
    address = Column(String)
    email = Column(String)
    
# Defines the Invoice class -> mapped to the invoices database table.
# Invoices class applies ForeignKey construct on custid attribute. 
# relationship(), tells the ORM that the Invoice class should be linked to the Customer class using the attribute Invoice.customer.
# The relationship() uses the foreign key relationships between the two tables to determine the nature of this linkage
class InvoiceX(Base2):
    __tablename__ = 'invoicesX'
    id = Column(Integer, primary_key = True)
    custid = Column(Integer, ForeignKey('customersX.id'))
    invno = Column(Integer)
    amount = Column(Integer)
    customerR = relationship("CustomerX", back_populates = "invoicesR")
    
CustomerX.invoicesR = relationship("InvoiceX", order_by = InvoiceX.id, back_populates = "customerR", cascade = "all, delete, delete-orphan")

engine2 = create_engine('sqlite:///Week7_NoSQL.db', echo=True)

Base2.metadata.create_all(engine2)

In [ ]:
# Add Data

from sqlalchemy.orm import sessionmaker

# Session class is defined using sessionmaker(), which is bound to the engine object created earlier
Session2 = sessionmaker(bind = engine2)

# session object (objSession) is set-up using the default constructor
sessionObj2 = Session2()

rows = [
    CustomerX(
      name = "Hello Neo4J", 
      address = "Lopez Halls, Exeter University",
      email = "Neo4J@gmail.com",
      invoicesR = [InvoiceX(invno = 3, amount = 10000), InvoiceX(invno = 4, amount = 5000)
    ]),
   CustomerX(
      name = "Hello Redis", 
      address = "Plymouth Quaeters, Plymouth", 
      email = "Redis@gmail.com", 
      invoicesR = [InvoiceX(invno = 7, amount = 12000), InvoiceX(invno = 8, amount = 18500)
    ]),
   CustomerX(
      name = "Hello Cassandra", 
      address = "L.12, Bristol University Computer Lab, Bristol BS1 6YU", 
      email = "Cassandra@gmail.com",
      invoicesR = [InvoiceX(invno = 9, amount = 15000), InvoiceX(invno = 11, amount = 6000)
   ])
]

sessionObj2.add_all(rows)
sessionObj2.commit()

In [ ]:
# To delete record from a single table, you have to delete an object of the mapped class from a session and commit the action.
recordSet = sessionObj2.query(CustomerX).get(3)

# Deleteing one record using .delete() method (pass object with primary key value = 2)
sessionObj2.delete(recordSet)

sessionObj2.commit()

# Confirm to see if deleted (count will be zero)
sessionObj2.query(CustomerX).filter_by(id = 2).count()

# However, related records present in Invoices
sessionObj2.query(InvoiceX).filter(InvoiceX.invno.in_([3,4])).count()

# 10. Creating Many-to-Many Relationship

In [ ]:
# Many to Many relationship between two tables is achieved by adding an association table such that it has two foreign keys - one from each table’s primary key. 
# Example used: Employee is a part of more than one department, and a department has more than one employee.

from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, ForeignKey, Table
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine

# Creates the Base class, which is what all models inherit from and how they get SQLAlchemy ORM functionality.
Base3=declarative_base()

# Define Employee and Department classes mapped to tables departmentTable and employeeTable 
class Department(Base3):
    __tablename__ = 'departmentTable'
    id = Column(Integer, primary_key = True)
    name = Column(String)
    employees = relationship('Employee', secondary = 'link')

class Employee(Base3):
    __tablename__ = 'employeeTable'
    id = Column(Integer, primary_key = True)
    name = Column(String)
    departments = relationship(Department,secondary = 'link')

# Define a Link class. It is linked to link table and contains department_id and employee_id attributes respectively referencing to primary keys of department and employee table.
class Link(Base3):
    __tablename__ = 'link'
    department_id = Column(Integer, ForeignKey('departmentTable.id'), primary_key = True)
    employee_id = Column(Integer, ForeignKey('employeeTable.id'), primary_key = True)
    

engine3 = create_engine('sqlite:///Week7_NoSQL.db', echo=True)
Base3.metadata.create_all(engine3)


In [ ]:
# Add data

# 1. Create objects of Department and Employee classes
d1 = Department(name = "Accounts")
d2 = Department(name = "Sales")
d3 = Department(name = "Marketing")
e1 = Employee(name = "John")
e2 = Employee(name = "Tony")
e3 = Employee(name = "Graham")

# 2. Each table has a collection attribute having append() method. We can add Department objects to Department collection of Employee object, and vice-versa
e1.departments.append(d1)
e2.departments.append(d3)
d1.employees.append(e3)
d2.employees.append(e2)
d3.employees.append(e1)
e3.departments.append(d2)

In [ ]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind = engine3)
sessionObj4 = Session()

# Add all objects to datbase and commit
sessionObj4.add(e1)
sessionObj4.add(e2)
sessionObj4.add(d1)
sessionObj4.add(d2)
sessionObj4.add(d3)
sessionObj4.add(e3)

sessionObj4.commit()

In [ ]:
# Display data from three tables - Many to Many.

for x in sessionObj4.query(Department, Employee).filter(Link.department_id == Department.id, 
   Link.employee_id == Employee.id).order_by(Link.department_id).all():
    print ("Department: {} Name: {}".format(x.Department.name, x.Employee.name))

# 11: Dropping Tables

In [ ]:
# All tables can be deleted using the drop_all() method. 
# This method does the exact opposite of create_all()
# When drop_all() command is issued, the presence of each table is checked first, and tables are dropped in reverse order of dependency.

Base3.metadata.drop_all(engine3)